In [1]:
from lib.utils_compress import *

In [2]:
import torch
import numpy as np

In [3]:
tv_state_dict = np.load("/root/autodl-fs/LLMEthicsPatches/task_vectors/gun-train_step-1000-operator---scale-1.0.npy", allow_pickle=True).item()

In [4]:
def quantize_per_tensor(weight, num_bits=8):
    qmin = -1.0 * (2**num_bits) / 2
    qmax = -1.0 * qmin - 1
    min_val, max_val = weight.min(), weight.max()

    scale = (max_val - min_val) / (qmax - qmin)
    zero_point = qmin - min_val / scale

    # 在这里，我们使用了numpy的round函数来实现四舍五入
    quantized_weight = torch.round(weight / scale + zero_point)

    # 我们需要确保量化后的权重在有效范围内
    quantized_weight = torch.clamp(quantized_weight, qmin, qmax).to(torch.int8)

    return quantized_weight, scale, zero_point

def dequantize_per_tensor(quantized_weight, scale, zero_point):
    return scale * (quantized_weight - zero_point)

In [5]:
quantized_tv_state_dict, scale_dict, zero_point_dict = quantize_task_vector(tv_state_dict)

In [6]:
#np.save("/root/autodl-fs/LLMEthicsPatches/task_vectors/gun-train_step-1000-operator---scale-1.0-quantized.npy", quantized_tv_state_dict)

In [7]:
pruned_quantized_tv_state_dict = prune_weights_ratio(quantized_tv_state_dict, 0.5)

In [10]:
np.save("/root/autodl-fs/LLMEthicsPatches/task_vectors/gun-train_step-1000-operator---scale-1.0-pruned+quantized.npy", pruned_quantized_tv_state_dict)

In [8]:
dequantized_tv_state_dict = dequantize_task_vector(pruned_quantized_tv_state_dict, scale_dict, zero_point_dict)

In [9]:
np.save("/root/autodl-fs/LLMEthicsPatches/task_vectors/gun-train_step-1000-operator---scale-1.0-pruned+dequantized.npy", dequantized_tv_state_dict)